## Install Dependencies

In [ ]:
!pip install tensorflow pandas matplotlib scikit-learn

: 

In [3]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np

Error: (1,10): error CS1002: ; expected
(2,15): error CS1002: ; expected
(2,15): error CS1525: Invalid expression term 'as'
(2,20): error CS1002: ; expected
(3,19): error CS1002: ; expected
(3,19): error CS1525: Invalid expression term 'as'
(3,24): error CS1002: ; expected
(4,14): error CS1002: ; expected
(4,14): error CS1525: Invalid expression term 'as'

In [ ]:
df = pd.read_csv(os.path.join("jigsaw-toxic-comment-classification-challenge" , "train.csv"))

In [ ]:
df

In [ ]:
df.iloc[5]['comment_text']

In [ ]:
df[df.columns[2:]].iloc[3]

## Preprocessing

In [ ]:
from tensorflow.keras.layers import TextVectorization

In [ ]:
X = df['comment_text']
y = df[df.columns[2:]].values

In [ ]:
MAX_FEATURES = 200000 # number of words in the vcab

In [ ]:
vectorizer = TextVectorization(max_tokens = MAX_FEATURES , output_sequence_length=1800,output_mode='int')

In [ ]:
vectorizer.adapt(X.values)

In [ ]:
vectorized_text = vectorizer(X.values)

In [ ]:
vectorized_text

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text , y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)

# MCSHBAP - map,cache , shuffle, batch , prefetch     from_tensor_slices , list_file

In [ ]:
batch_X , batch_y = dataset.as_numpy_iterator().next()

In [ ]:
batch_X.shape

In [ ]:
batch_y

In [ ]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*0.7)).take(int(len(dataset) * 0.2))
test = dataset.skip(int(len(dataset) * 0.9)).take(int(len(dataset) * 0.1 ))

## Create Deep neural network

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM , Dropout , Bidirectional,Dense , Embedding

In [ ]:
model = Sequential()

In [ ]:
model.add(Embedding(MAX_FEATURES+1 , 32 , input_shape=(1800,)))
model.add(Bidirectional(LSTM(32 , activation = 'tanh')))

model.add(Dense(128 , activation = 'relu'))
model.add(Dense(256 , activation = 'relu'))
model.add(Dense(128 , activation = 'relu'))
model.add(Dense(6 , activation = 'sigmoid'))

In [ ]:
model.compile(optimizer='adam',
              loss='BinaryCrossentropy',  # Suitable for multi-label classification
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(train , epochs = 1,validation_data = val)

In [ ]:
history.history

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.figure(figsize=(8,5))
pd.DataFrame(history.history).plot()
plt.show()

## Make Predictions

In [ ]:
input_text = vectorizer('You freaking suck! I am going to hit you.')

In [ ]:
res = model.predict(input_text)

In [ ]:
(res > 0.5).astype(int)

In [ ]:
batch_X, batch_y = test.as_numpy_iterator().next()

In [ ]:
(model.predict(batch_X) > 0.5).astype(int)
res.shape

## Evaluate Model

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

In [ ]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [ ]:
for batch in test.as_numpy_iterator(): 
    # Unpack the batch 
    X_true, y_true = batch
    # Make a prediction 
    yhat = model.predict(X_true)
    
    # Flatten the predictions
    y_true = y_true.flatten()
    yhat = yhat.flatten()
    
    pre.update_state(y_true, yhat)
    re.update_state(y_true, yhat)
    acc.update_state(y_true, yhat)

In [ ]:
print(f'Precision: {pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')

##  Test and Gradio

In [ ]:
!pip install gradio jinja2

In [ ]:
import tensorflow as tf
import gradio as gr

model.save('toxicity.h5')

model = tf.keras.models.load_model('toxicity.h5')

input_str = vectorizer('hey i freaken hate you!')


res = model.predict(np.expand_dims(input_str,0))

In [ ]:
res

In [ ]:
def score_comment(comment):
    vectorized_comment = vectorizer([comment])
    results = model.predict(vectorized_comment)
    
    text = ''
    for idx, col in enumerate(df.columns[2:]):
        text += '{}: {}\n'.format(col, results[0][idx]>0.5)
    
    return text

In [ ]:
interface = gr.Interface(fn=score_comment, 
                         inputs=gr.inputs.Textbox(lines=2, placeholder='Comment to score'),
                        outputs='text')

In [ ]:
interface.launch(share=True)